# Portfolio Risk Analysis - Visualizations

This notebook creates interactive visualizations for the Monte Carlo Portfolio Risk Simulation.

In [ ]:
# Import libraries
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import project modules
from src.data.loader import DataLoader
from src.portfolio.construction import Portfolio
from src.simulation.monte_carlo import MonteCarloSimulator
from src.risk.metrics import RiskMetrics

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

In [ ]:
# Generate sample data (for demonstration)
from demo import generate_simulated_data
from src.utils.config import Config

# Load or generate data
prices, returns = generate_simulated_data(Config.DEFAULT_ETFS, n_days=1260)
portfolio = Portfolio(Config.DEFAULT_ETFS, notional_value=100000)

print(f"Data loaded: {returns.shape[0]} days, {returns.shape[1]} assets")
print(f"Date range: {returns.index.min().date()} to {returns.index.max().date()}")

## 1. Portfolio Price Evolution

In [ ]:
# Plot individual asset prices
fig = go.Figure()

for ticker in Config.DEFAULT_ETFS:
    fig.add_trace(go.Scatter(
        x=prices.index,
        y=prices[ticker],
        mode='lines',
        name=ticker,
        line=dict(width=2)
    ))

fig.update_layout(
    title='ETF Price Evolution Over Time',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    hovermode='x unified',
    template='plotly_white',
    height=600
)

fig.show()

In [ ]:
# Portfolio value evolution
portfolio_values = portfolio.calculate_portfolio_values(returns)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=portfolio_values.index,
    y=portfolio_values,
    mode='lines',
    name='Portfolio Value',
    line=dict(color='blue', width=3)
))

fig.update_layout(
    title='Portfolio Value Evolution ($100K Initial)',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    template='plotly_white',
    height=500
)

fig.show()

## 2. Correlation Heatmap

In [ ]:
# Correlation matrix heatmap
corr_matrix = returns.corr()

fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='Asset Correlation Matrix',
    aspect='auto'
)

fig.update_layout(
    height=600,
    template='plotly_white'
)

fig.show()

## 3. Monte Carlo Simulation Results

In [ ]:
# Run Monte Carlo simulation
simulator = MonteCarloSimulator(
    returns=returns,
    portfolio_weights=portfolio.get_weights_array(),
    random_seed=42
)

# Generate simulations
n_sims = 50000
pnl_sims = simulator.simulate_portfolio_pnl(100000, n_sims)
return_sims = pnl_sims / 100000  # Convert to returns

print(f"Generated {n_sims:,} Monte Carlo simulations")

In [ ]:
# Plot P&L distribution
fig = go.Figure()

# Histogram
fig.add_trace(go.Histogram(
    x=pnl_sims,
    nbinsx=100,
    name='P&L Distribution',
    opacity=0.7,
    marker_color='lightblue'
))

# Add VaR lines
var_95 = np.percentile(pnl_sims, 5)
var_99 = np.percentile(pnl_sims, 1)

fig.add_vline(x=var_95, line_dash="dash", line_color="orange", 
              annotation_text=f"95% VaR: ${var_95:,.0f}")
fig.add_vline(x=var_99, line_dash="dash", line_color="red", 
              annotation_text=f"99% VaR: ${var_99:,.0f}")

fig.update_layout(
    title='Monte Carlo P&L Distribution (50,000 simulations)',
    xaxis_title='Daily P&L ($)',
    yaxis_title='Frequency',
    template='plotly_white',
    height=500
)

fig.show()

## 4. Risk Metrics Dashboard

In [ ]:
# Create subplots for risk metrics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('VaR vs CVaR', 'Return Distribution', 'Drawdown', 'Rolling Volatility'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. VaR vs CVaR comparison
confidence_levels = [90, 95, 99]
vars = [np.percentile(pnl_sims, 100-cl) for cl in confidence_levels]
cvars = [np.mean(pnl_sims[pnl_sims <= np.percentile(pnl_sims, 100-cl)]) for cl in confidence_levels]

fig.add_trace(
    go.Bar(x=confidence_levels, y=vars, name='VaR', marker_color='orange'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=confidence_levels, y=cvars, name='CVaR', marker_color='red'),
    row=1, col=1
)

# 2. Return distribution
portfolio_returns = portfolio.calculate_portfolio_returns(returns)
fig.add_trace(
    go.Histogram(x=portfolio_returns*100, nbinsx=50, name='Returns', marker_color='blue'),
    row=1, col=2
)

# 3. Drawdown
cumulative = (1 + portfolio_returns).cumprod()
running_max = cumulative.expanding().max()
drawdown = (cumulative - running_max) / running_max * 100

fig.add_trace(
    go.Scatter(x=drawdown.index, y=drawdown, mode='lines', name='Drawdown', 
               fill='tonexty', fillcolor='rgba(255,0,0,0.3)', line_color='red'),
    row=2, col=1
)

# 4. Rolling volatility
rolling_vol = portfolio_returns.rolling(window=21).std() * np.sqrt(252) * 100
fig.add_trace(
    go.Scatter(x=rolling_vol.index, y=rolling_vol, mode='lines', 
               name='21-day Rolling Vol', line_color='green'),
    row=2, col=2
)

fig.update_layout(
    title_text="Portfolio Risk Metrics Dashboard",
    height=800,
    template='plotly_white'
)

fig.show()

## 5. Portfolio Composition

In [ ]:
# Portfolio weights pie chart
fig = px.pie(
    values=list(portfolio.weights.values()),
    names=list(portfolio.weights.keys()),
    title='Portfolio Allocation'
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label'
)

fig.update_layout(
    template='plotly_white',
    height=500
)

fig.show()

## 6. Risk-Return Scatter Plot

In [ ]:
# Calculate risk-return metrics for each asset
asset_metrics = []

for ticker in Config.DEFAULT_ETFS:
    asset_returns = returns[ticker]
    annual_return = asset_returns.mean() * 252
    annual_vol = asset_returns.std() * np.sqrt(252)
    sharpe = annual_return / annual_vol if annual_vol > 0 else 0
    
    asset_metrics.append({
        'Asset': ticker,
        'Return': annual_return * 100,
        'Volatility': annual_vol * 100,
        'Sharpe': sharpe
    })

metrics_df = pd.DataFrame(asset_metrics)

# Risk-return scatter plot
fig = px.scatter(
    metrics_df,
    x='Volatility',
    y='Return',
    size='Sharpe',
    color='Asset',
    title='Risk-Return Profile by Asset',
    labels={'Volatility': 'Annual Volatility (%)', 'Return': 'Annual Return (%)'}
)

# Add portfolio point
portfolio_return = portfolio_returns.mean() * 252 * 100
portfolio_vol = portfolio_returns.std() * np.sqrt(252) * 100

fig.add_trace(go.Scatter(
    x=[portfolio_vol],
    y=[portfolio_return],
    mode='markers',
    marker=dict(size=15, color='black', symbol='star'),
    name='Portfolio'
))

fig.update_layout(
    template='plotly_white',
    height=600
)

fig.show()

In [ ]:
# Print summary statistics
print("=" * 60)
print("PORTFOLIO RISK ANALYSIS SUMMARY")
print("=" * 60)
print(f"Portfolio Value: ${portfolio.notional_value:,.2f}")
print(f"Annual Return: {portfolio_return:.2f}%")
print(f"Annual Volatility: {portfolio_vol:.2f}%")
print(f"Sharpe Ratio: {(portfolio_return/100) / (portfolio_vol/100):.3f}")
print(f"Maximum Drawdown: {drawdown.min():.2f}%")
print(f"95% VaR (1-day): ${var_95:,.2f}")
print(f"99% VaR (1-day): ${var_99:,.2f}")
print("=" * 60)